In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 10


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
1,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
2,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
3,TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.427048,-76.547928
4,TANGARA_2E9A,d29ed62,D29ESP32DED2E9A,3.4407806396484375 -76.54106140136719,3.440781,-76.541061


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-11-09T00:00:00-05:00 -> 1699506000000, Until: 2023-11-10T23:59:59-05:00 -> 1699678799000
Group by Time: 30s


In [4]:
# PM2.5 Data Frame Sensors
df_pm25_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'pm25', GROUP_BY_TIME)
df_pm25_raw.head()
#df_pm25_raw.shape

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,
2023-11-09 00:00:00-05:00,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,9.0,NaN
2023-11-09 00:00:30-05:00,11.0,12.0,NaN,9.0,3.0,11.0,NaN,3.0,10.0,12.0
2023-11-09 00:01:00-05:00,12.0,12.0,10.0,9.0,2.0,13.0,NaN,6.0,10.0,14.0
2023-11-09 00:01:30-05:00,12.0,12.0,10.0,9.0,3.0,13.0,NaN,6.0,10.0,14.0
2023-11-09 00:02:00-05:00,11.0,13.0,10.0,10.0,2.0,13.0,NaN,9.0,10.0,14.0


## Descriptive Statistics

In [5]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5609.000000,5639.000000,3836.000000,4181.000000,5666.000000,5683.000000,3226.000000,5506.000000,5723.000000,5522.000000
mean,9.393475,11.711296,12.083942,7.288448,6.432227,7.373394,7.687539,6.244279,9.574699,9.730170
std,4.733606,7.959368,6.522791,4.396990,8.252764,3.117201,2.824535,3.828438,4.974644,4.887244
min,1.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,6.000000,8.000000,8.000000,3.000000,3.000000,5.000000,6.000000,4.000000,6.000000,6.000000
50%,9.000000,9.000000,11.000000,8.000000,6.000000,8.000000,8.000000,6.000000,8.000000,10.000000
75%,12.000000,12.000000,14.000000,10.000000,9.000000,9.000000,9.000000,8.000000,12.000000,12.000000
max,27.000000,76.000000,53.000000,23.000000,527.000000,17.000000,43.000000,27.000000,73.000000,94.000000


## Missing Data

In [6]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 67%, Missing: 33%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 73%, Missing: 27%, To be checked
Tangara Sensor: TANGARA_307A, Data: 56%, Missing: 44%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_raw, "pm25_raw.csv")